<a href="https://colab.research.google.com/github/gozdeaslantas/NLP_Learning/blob/master/roberta_nlbse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

def connect_drive():
  drive.mount('/content/gdrive', force_remount=True)

connect_drive()

Mounted at /content/gdrive


In [ ]:
!pip install transformers --quiet
!pip install sentencepiece --quiet
!pip install textaugment

from transformers import XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
import copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import sklearn.metrics
from sklearn.preprocessing import LabelEncoder
import gensim
import re
import gc
from sklearn.metrics import roc_auc_score, accuracy_score
torch.cuda.empty_cache()

     |████████████████████████████████| 3.5 MB 31.9 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 6.8 MB 45.0 MB/s 
     |████████████████████████████████| 895 kB 15.4 MB/s 
     |████████████████████████████████| 596 kB 59.7 MB/s 
     |████████████████████████████████| 1.2 MB 29.6 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 1.3 MB 60.6 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 53 kB 2.9 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=27cb2e3858d5c5ca333fe674eedb615ce8eb56450d391108d5dd660cd4aa8eca
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans


In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import random as rd
#required libraries for wordnet based text aug.
try:
  import textaugment
except ModuleNotFoundError:
  !pip install textaugment
  import textaugment
  
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import pandas as pd
import os
# download the training set if it does not exist
if not os.path.isfile("github-labels-top3-803k-train.csv"):
  !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-train.tar.gz" | tar -xz
  
trainset = pd.read_csv("github-labels-top3-803k-train.csv")
trainset.groupby("issue_label").size()
trainset.head(2).T

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.security.selinux'
100  236M  100  236M    0     0  10.2M      0  0:00:23  0:00:23 --:--:-- 10.1M


,0,1
Unnamed: 0,0,1
issue_url,https://api.github.com/repos/eamodio/vscode-gi...,https://api.github.com/repos/binwiederhier/pco...
issue_label,bug,bug
issue_created_at,2021-01-02T18:07:30Z,2020-12-31T18:19:31Z
issue_author_association,NONE,OWNER
repository_url,https://api.github.com/repos/eamodio/vscode-gi...,https://api.github.com/repos/binwiederhier/pcopy
issue_title,Welcome screen on every editor window is very ...,"""pcopy invite"" and ""pcopy paste abc:"" does not..."
issue_body,I just discovered Gitlens and find the functio...,NaN


In [ ]:
if not os.path.isfile("github-labels-top3-803k-test.csv"):
  !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-test.tar.gz" | tar -xz

testset = pd.read_csv("github-labels-top3-803k-test.csv")
testset.groupby("issue_label").size()


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.security.selinux'
100 27.2M  100 27.2M    0     0  8022k      0  0:00:03  0:00:03 --:--:-- 8020k


issue_label
bug            40288
enhancement    33203
question        7027
dtype: int64

In [ ]:
from textaugment import Wordnet


def apply_word_net_based_augmentation(df, existing_len, target_len):
  v = True # enable verbs augmentation. By default is True.
  n = True # enable nouns augmentation. By default is False.
  runs = 1 # number of times to augment a sentence. By default is 1.
  p = 0.5 # The probability of success of an individual trial. (0.1<p<1.0), default is 0.5. Used by Geometric distribution to selects words from a sentence.

  #augment train data with class = 1 (Nonpersonal_reports)
  diff = target_len-existing_len
  to_append = []
  for i in range(diff):
    index = rd.randint(0, existing_len-1)
    t = Wordnet(v=True ,n=True, p=0.5)
    generated_sentence = t.augment(df['combined'][index])
    to_append.append([generated_sentence, df['issue_label'][index]])
    
  #df_to_append = pd.DataFrame(to_append, index = range(len_df, len_df+size_to_augment_label_one))
  df_to_append = pd.DataFrame(to_append, columns = df.columns.values, index = range(existing_len, target_len))
  df = pd.concat([df, df_to_append], ignore_index = False)
  df = df.sort_index().reset_index(drop=True)

  return df

In [ ]:
def equalize_class_distribution(train_df):
    count_bug, count_en, count_quest = train_df.issue_label.value_counts()
    print(count_bug, count_en, count_quest)
    # Divide by class
    df_bug = train_df[train_df['issue_label'] == 0]
    df_bug.reset_index(drop=True, inplace=True)
    df_enhancement = train_df[train_df['issue_label'] == 1]
    df_enhancement.reset_index(drop=True, inplace=True)
    df_question = train_df[train_df['issue_label'] == 2]
    df_question.reset_index(drop=True, inplace=True)

    df_bug_under = df_bug.sample(count_en)
    df_question_augment = apply_word_net_based_augmentation(df_question, count_quest, count_en)

    print(df_bug_under.issue_label.value_counts(), 
        df_enhancement.issue_label.value_counts(), df_question_augment.issue_label.value_counts())

    train_df = pd.concat([df_bug_under, df_enhancement, df_question_augment], axis=0)
  
    print(train_df.shape)
	
    return train_df

In [ ]:
# preprocessing can be customized by participants
def preprocess(row):
	# concatenate title and body, then remove whitespaces
	doc = ""
	doc += str(row.issue_title)
	doc += " "
	doc += str(row.issue_body)
	# https://radimrehurek.com/gensim/parsing/preprocessing.html
	doc = gensim.parsing.preprocessing.strip_multiple_whitespaces(doc)
	doc = re.sub("[^a-zA-Z' ]+", ' ', doc)
	return doc

In [ ]:
def preprocess_test_and_train():
    mlb = LabelEncoder()
	#trainset = pd.read_csv("github-labels-top3-803k-train.csv")
	#testset = pd.read_csv("github-labels-top3-803k-test.csv")

	# rng used to split training set into fasttext hyperparameter finetuning validation set (2 of 9 parts)
    random = np.random.default_rng(0)
    new_combined_column = []
    for row in tqdm(trainset.itertuples(), desc="Transform to fastText format", total=len(trainset)):
	    doc = f"{preprocess(row)}\n"
	    new_combined_column.append(doc)
     
  #merge text columns and transform labels
    trainset["combined"] = new_combined_column
    trainset["issue_label"] = mlb.fit_transform(trainset["issue_label"])
	
 
    df_train = copy.deepcopy(trainset[['combined', 'issue_label']])
    df_train = equalize_class_distribution(df_train)

    new_combined_column = []
    for row in tqdm(testset.itertuples(), desc="Transform to fastText format", total=len(testset)):
	    doc = f"{preprocess(row)}\n"
	    new_combined_column.append(doc)
    testset["combined"] = new_combined_column
    testset["issue_label"] = mlb.fit_transform(testset["issue_label"])

    df_test = copy.deepcopy(testset[['combined', 'issue_label']])

    return df_train, df_test

In [ ]:
df_train, df_test = preprocess_test_and_train()
df_train=df_train[df_train.combined.apply(lambda x: len(str(x))<512)]
df_train.reset_index(drop=True, inplace=True)
df_test=df_test[df_test.combined.apply(lambda x: len(str(x))<512)]
df_test.reset_index(drop=True, inplace=True)

Transform to fastText format: 100%|██████████| 722899/722899 [01:10<00:00, 10194.91it/s]


361103 299374 62422
0    299374
Name: issue_label, dtype: int64 1    299374
Name: issue_label, dtype: int64 2    299374
Name: issue_label, dtype: int64
(898122, 2)


Transform to fastText format: 100%|██████████| 80518/80518 [00:07<00:00, 10378.28it/s]


In [ ]:
#create dataframe with rows which include text longer than 512
#df_train2 = copy.deepcopy(df_train[['combined', 'issue_label']])
#print('mevcut: ', df_train2.shape)
#df_train2=df_train2[df_train2.combined.apply(lambda x: len(str(x))>512)]
#print('512 den uzun',df_train.shape)
#print('512 den uzun ilk cümle: ', df_train2['combined'].iloc[0])
#print(len(df_train2['combined'].iloc[0]))

#df_train['combined'] = df_train['combined'].apply(lambda x: " ".join(str(x).split()[:512])) 
#print('ilk 512',df_train.shape)
#print('ilk 512: ', df_train['combined'].iloc[0])
#print(len(df_train['combined'].iloc[0]))

#df_train2['combined'] = df_train2['combined'].apply(lambda x: " ".join(str(x).split()[512:1024]))  
#print(df_train2.shape)
#print('512 den uzun ilk cümlenin devamı: ', df_train2['combined'].iloc[0])

#merged_train = pd.concat([df_train, df_train2], axis=0)
#merged_train.reset_index(drop=True, inplace=True)
#print(merged_train.shape)

In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
# EPOCHS = 1
LEARNING_RATE = 1e-05
NUM_CORES = os.cpu_count()
print(NUM_CORES)
xlmr_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
xlmr_model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3)

2


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [ ]:
class IssueData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.combined
        self.targets = self.data.issue_label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
training_set = IssueData(df_train, xlmr_tokenizer, MAX_LEN)
testing_set = IssueData(df_test, xlmr_tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': NUM_CORES
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': NUM_CORES
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
#model = RobertaClass()
xlmr_model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [ ]:
#Fine tune model
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(params =  xlmr_model.parameters(), lr=LEARNING_RATE, eps = 1e-8)

def calcuate_accuracy(preds, targets, batch_index, targets_list, stacked_val_preds):
    val_preds = preds.detach().cpu().numpy()
    targets_np = targets.to('cpu').numpy()
    targets_list.extend(targets_np)

    if batch_index == 0:  # first batch
        stacked_val_preds = val_preds
    else:
        stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

    
    # Calculate the validation accuracy
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    acc = accuracy_score(y_true, y_pred)
    return acc

In [ ]:
def train(EPOCHS):
  for epoch in range(EPOCHS):
    total_train_loss = 0
    stacked_tr_preds = []
    targets_list = []
    acc = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    xlmr_model.train()
    for batch_index, data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        labels = data['labels'].to(device, dtype = torch.long)

        xlmr_model.zero_grad() 
        outputs = xlmr_model(ids, attention_mask=mask, labels=labels)
        loss = outputs[0] #loss_function(outputs, targets)
        total_train_loss += loss.item()
        #tr_loss += loss.item()
        
        #big_val, big_idx = torch.max(outputs[1], dim=1)
        #acc = calcuate_accuracy(outputs[1], labels, batch_index, targets_list, stacked_tr_preds)

        nb_tr_steps += 1
        nb_tr_examples+=labels.size(0)
        
        #if batch_index%5000==0:
        #    loss_step = total_train_loss/nb_tr_steps
        #    accu_step = (acc*100)/nb_tr_examples 
        #    print(f"Training Loss per 5000 steps: {loss_step}")
        #    print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(xlmr_model.parameters(), 1.0)
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(acc*100)/nb_tr_examples}')
    epoch_loss = total_train_loss/nb_tr_steps
    epoch_accu = (acc*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    torch.save(xlmr_model.state_dict(), "model-{}".format(epoch))
    gc.collect()
  return 

In [ ]:
torch.cuda.empty_cache()
gc.collect()
EPOCHS = 15
train(EPOCHS)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to th

RuntimeError: ignored

In [ ]:
def predict():
  print("device: ", device)

  mdl = xlmr_model.to(device)
  mdl.eval()

  y_pred = []
  y_true = []

  stacked_preds = []
  targets_list = []
  for batch_index, data in tqdm(enumerate(testing_loader, 0)):

      test_label = data['labels'].to(device, dtype = torch.long)
      mask = data['mask'].to(device)
      input_id = data['ids'].squeeze(1).to(device)

      outputs = mdl(input_id, mask)
      preds = outputs[0]
      
      # Move preds to the CPU
      preds = preds.detach().cpu().numpy()
        
        # Move the labels to the cpu
      targets_np = test_label.to('cpu').numpy()

        # Append the labels to a numpy list
      targets_list.extend(targets_np)

      if batch_index == 0:  # first batch
        stacked_preds = preds
      else:
        stacked_preds = np.vstack((stacked_preds, preds))


  y_true=targets_list
  y_pred = np.argmax(stacked_preds, axis=1)
  print(len(y_pred))
  return y_true, y_pred


In [ ]:
import sklearn.metrics

def evaluation(y_true, y_pred):
  for label in [0, 1, 2]:
    P_c = sklearn.metrics.precision_score(y_true, y_pred, average=None, labels=[label])[0]
    R_c = sklearn.metrics.recall_score(y_true, y_pred, average=None, labels=[label])[0]
    F1_c = sklearn.metrics.f1_score(y_true, y_pred, average=None, labels=[label])[0]
    print(f"=*= {label} =*=")
    print(f"precision:\t{P_c:.4f}")
    print(f"recall:\t\t{R_c:.4f}")
    print(f"F1 score:\t{F1_c:.4f}")
    print()


  P = sklearn.metrics.precision_score(y_true, y_pred, average='micro')
  R = sklearn.metrics.recall_score(y_true, y_pred, average='micro')
  F1 = sklearn.metrics.f1_score(y_true, y_pred, average='micro')

  print("=*= global =*=")
  print(f"precision:\t{P:.4f}")
  print(f"recall:\t\t{R:.4f}")
  print(f"F1 score:\t{F1:.4f}")

In [ ]:
y_true, y_pred = predict()
evaluation(y_true, y_pred)

In [ ]:
output_model_file = 'gdrive/MyDrive/bitsin/pytorch_roberta_sentiment.bin'
output_vocab_file = 'gdrive/MyDrive/'

model_to_save = xlmr_model
torch.save(model_to_save, output_model_file)
xlmr_tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')